In [8]:
# --------------------------------
# 0) Imports
# --------------------------------
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)


RANDOM_STATE = 42  # reproducibility

In [3]:
# --------------------------------
# 1) Load the dataset
# --------------------------------
# Expected columns: "Category" (ham/spam), "Message" (text)
df = pd.read_csv("mail_l7_dataset.csv")

#preprocessing data

# Basic cleaning: replace NaNs with empty strings (text models can't handle NaN)
df = df.where(pd.notnull(df), "")

# Encode labels: spam -> 0, ham -> 1  (keep your original convention)
df.loc[df["Category"].str.lower().str.strip() == "spam", "Category"] = 0
df.loc[df["Category"].str.lower().str.strip() == "ham",  "Category"] = 1

print(df.head())

  Category                                            Message
0        1  Go until jurong point, crazy.. Available only ...
1        1                      Ok lar... Joking wif u oni...
2        0  Free entry in 2 a wkly comp to win FA Cup fina...
3        1  U dun say so early hor... U c already then say...
4        1  Nah I don't think he goes to usf, he lives aro...


In [5]:
# --------------------------------
# 2) Split features (X) and target (y)
# --------------------------------
X = df["Message"].astype(str)
y = df["Category"].astype(int)

# --------------------------------
# 3) Train/test split (stratified)
# --------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print("=== SPLIT SIZES ===")
print("Train:", X_train.shape[0], " | Test:", X_test.shape[0])

=== SPLIT SIZES ===
Train: 4457  | Test: 1115


In [6]:
# --------------------------------
# 4) Text → TF-IDF features
# --------------------------------
# Simple, student-friendly settings
tfidf = TfidfVectorizer(min_df=1, stop_words="english", lowercase=True)
X_train_features = tfidf.fit_transform(X_train)
X_test_features  = tfidf.transform(X_test)

print("\n=== TF-IDF SHAPES ===")
print("X_train:", X_train_features.shape, " | X_test:", X_test_features.shape)


=== TF-IDF SHAPES ===
X_train: (4457, 7440)  | X_test: (1115, 7440)


In [10]:
# --------------------------------
# 5) Train Logistic Regression (baseline)
# --------------------------------
lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
lr.fit(X_train_features, y_train)
lr_pred = lr.predict(X_test_features)

# --------------------------------
# 6) Train Random Forest
#    (convert TF-IDF to dense for tree models)
# --------------------------------
rf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
rf.fit(X_train_features, y_train)
rf_pred = rf.predict(X_test_features.toarray())

# --------------------------------
# 7) Train Naive Bayes (Multinomial)
# --------------------------------
nb = MultinomialNB()
nb.fit(X_train_features, y_train)
nb_pred = nb.predict(X_test_features)

In [11]:
# --------------------------------
# 8) Helper functions: metrics + confusion matrix print
# --------------------------------
def print_clf_metrics(name, y_true, y_pred, pos_label=0):
    """Print Accuracy, Precision, Recall, F1. pos_label=0 means 'spam' is positive."""
    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, pos_label=pos_label)
    rec  = recall_score(y_true, y_pred, pos_label=pos_label)
    f1   = f1_score(y_true, y_pred, pos_label=pos_label)
    print(f"\n{name} Performance:")
    print(f"  Accuracy : {acc:.3f}")
    print(f"  Precision: {prec:.3f}  (positive = spam=0)")
    print(f"  Recall   : {rec:.3f}  (positive = spam=0)")
    print(f"  F1-Score : {f1:.3f}  (positive = spam=0)")

def print_confmat(name, y_true, y_pred):
    """
    Confusion matrix with readable labels.
    Rows = Actual, Cols = Predicted
    Order: [Ham(1), Spam(0)] so you can see both classes clearly.
    """
    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
    cm_df = pd.DataFrame(
        cm,
        index   = ["Actual: Ham (1)",  "Actual: Spam (0)"],
        columns = ["Pred: Ham (1)",    "Pred: Spam (0)"]
    )
    print(f"\n{name} – Confusion Matrix:\n{cm_df}")

# --------------------------------
# 9) Show results for both models (same print style as L5)
# --------------------------------
print_clf_metrics("Logistic Regression", y_test, lr_pred, pos_label=0)
print_confmat("Logistic Regression", y_test, lr_pred)

print_clf_metrics("Random Forest", y_test, rf_pred, pos_label=0)
print_confmat("Random Forest", y_test, rf_pred)

print_clf_metrics("Naive Bayes", y_test, nb_pred, pos_label=0)
print_confmat("Naive Bayes", y_test, nb_pred)


Logistic Regression Performance:
  Accuracy : 0.968
  Precision: 1.000  (positive = spam=0)
  Recall   : 0.758  (positive = spam=0)
  F1-Score : 0.863  (positive = spam=0)

Logistic Regression – Confusion Matrix:
                  Pred: Ham (1)  Pred: Spam (0)
Actual: Ham (1)             966               0
Actual: Spam (0)             36             113

Random Forest Performance:
  Accuracy : 0.983
  Precision: 1.000  (positive = spam=0)
  Recall   : 0.872  (positive = spam=0)
  F1-Score : 0.932  (positive = spam=0)

Random Forest – Confusion Matrix:
                  Pred: Ham (1)  Pred: Spam (0)
Actual: Ham (1)             966               0
Actual: Spam (0)             19             130

Naive Bayes Performance:
  Accuracy : 0.977
  Precision: 1.000  (positive = spam=0)
  Recall   : 0.826  (positive = spam=0)
  F1-Score : 0.904  (positive = spam=0)

Naive Bayes – Confusion Matrix:
                  Pred: Ham (1)  Pred: Spam (0)
Actual: Ham (1)             966               0
Ac

In [13]:
# --------------------------------
# 10) Sanity Checks – 3 custom messages
# --------------------------------
samples = [
    "Free entry in 2 a weekly competition!",
    "I will meet you at the cafe tomorrow",
    "Congratulations, you won a free ticket"
]

print("\n=== SANITY CHECKS ON CUSTOM MESSAGES ===")
for msg in samples:
    msg_tfidf = tfidf.transform([msg])
    lr_pred = int(lr.predict(msg_tfidf)[0])
    rf_pred = int(rf.predict(msg_tfidf.toarray())[0])
    nb_pred = int(nb.predict(msg_tfidf)[0])

    def lab2str(v):
        return "Spam (0)" if v == 0 else "Ham (1)"

    print(f"\nMessage: {msg}")
    print(f"  Logistic Regression: {lab2str(lr_pred)}")
    print(f"  Random Forest      : {lab2str(rf_pred)}")
    print(f"  Naive Bayes        : {lab2str(nb_pred)}")



=== SANITY CHECKS ON CUSTOM MESSAGES ===

Message: Free entry in 2 a weekly competition!
  Logistic Regression: Ham (1)
  Random Forest      : Ham (1)
  Naive Bayes        : Spam (0)

Message: I will meet you at the cafe tomorrow
  Logistic Regression: Ham (1)
  Random Forest      : Ham (1)
  Naive Bayes        : Ham (1)

Message: Congratulations, you won a free ticket
  Logistic Regression: Ham (1)
  Random Forest      : Ham (1)
  Naive Bayes        : Ham (1)
